In [11]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
from google.colab import drive
drive.mount('/content/drive')
model = load_model('/content/drive/MyDrive/LSTM_Next_Word_Prediction_Model.h5')

# Load the tokenizer (saved previously using pickle)
with open('/content/drive/MyDrive/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# Get max sequence length from model input shape
max_seq_len = model.input_shape[1]

# --- Predict Next Word ---
def predict_next_word(input_text, num_words=1):
    text = input_text.lower().strip()
    for _ in range(num_words):
        # Tokenize
        token_list = tokenizer.texts_to_sequences([text])[0]
        token_list = token_list[-(max_seq_len - 1):]  # keep the last n tokens

        # Pad
        padded = pad_sequences([token_list], maxlen=max_seq_len - 1, padding='pre')

        # Predict
        predicted_probs = model.predict(padded, verbose=0)
        predicted_index = np.argmax(predicted_probs)

        # Map back to word
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                text += ' ' + word
                break
    return text
